In [2]:
import pandas as pd
import numpy as np

import mysql.connector
from sqlalchemy import create_engine
import pandas.io.sql as psql
import sqlalchemy

from dateutil import parser
from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta


In [3]:
database_username = 'root'
database_password = 'yash'
database_ip       = '127.0.0.1'
database_name     = 'mainschema'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))

In [4]:
df = pd.read_sql('SELECT * FROM marketsmojorecos', con=database_connection)
del df['srno']
del df['index']


In [5]:
df

,broker,company_name,current_price,recomended_buying,predict_date,target_price
0,Prabhudas Lilladher,Colgate Palmol. (I),1248.25,1274.35,2019-09-03,1159.00
1,Axis Direct,Larsen & Toubro,1310.90,1328.1,2019-09-03,1487.00
2,Nirmal Bang,Britannia Inds,2695.00,2704.8,2019-09-03,3220.00
3,Sushil Finance,Sahyadri Industries,147.00,145.5,2019-09-03,220.00
4,Geojit Financial Services,Tata Chemicals,580.95,579.06,2019-08-30,531.00
...,...,...,...,...,...,...
22115,Prabhudas Lilladher,BPCL,493.00,378.64,2019-09-05,466.00
22116,Axis Direct,Bajaj Finance,4056.35,3243.58,2019-09-04,3591.00
22117,Geojit Financial Services,Hindalco,207.60,179.21,2019-09-04,195.00
22118,Geojit Financial Services,HDFC,2368.00,2096.68,2019-09-04,2284.00


In [6]:
df.drop_duplicates(inplace = True)
df

,broker,company_name,current_price,recomended_buying,predict_date,target_price
0,Prabhudas Lilladher,Colgate Palmol. (I),1248.25,1274.35,2019-09-03,1159.00
1,Axis Direct,Larsen & Toubro,1310.90,1328.1,2019-09-03,1487.00
2,Nirmal Bang,Britannia Inds,2695.00,2704.8,2019-09-03,3220.00
3,Sushil Finance,Sahyadri Industries,147.00,145.5,2019-09-03,220.00
4,Geojit Financial Services,Tata Chemicals,580.95,579.06,2019-08-30,531.00
...,...,...,...,...,...,...
21738,Motilal Oswal,Multi Commodity Exch,1122.00,864.94,2019-09-05,1000.00
21739,Axis Direct,Bajaj Finance,4056.35,3243.58,2019-09-04,3591.00
21740,Geojit Financial Services,Hindalco,207.60,179.21,2019-09-04,195.00
21741,Geojit Financial Services,HDFC,2368.00,2096.68,2019-09-04,2284.00


In [7]:
start_date = '2018-01-01'
end_date = '2018-10-31'
date_object1 = datetime.strptime(start_date, '%Y-%m-%d').date()
date_object2 = datetime.strptime(end_date, '%Y-%m-%d').date()

In [8]:
mask = (df['predict_date'] > date_object1) & (df['predict_date'] <= date_object2)

# 2018 data

In [9]:
df1 = df.loc[mask]
df1 = df1.reset_index()
#del df1['level_0']
#del df1['srno']
del df1['index']

In [10]:
print("No of Brokers in 2018 :",df1['broker'].nunique()) 
s1=len(df1)

No of Brokers in 2018 : 50


In [11]:
df3=pd.DataFrame()
for i in range(0,len(df1)): 
    sym=df1.at[i, 'company_name']
    sym = sym.lower()
    sym = sym.replace(" ","")
    sym = sym.replace(".","")
    sym = sym.replace("(i)","")
    sym = sym.replace("(","")
    sym = sym.replace(")","")
    sym = sym.replace("-","")
    sqlrun = "SELECT SYMBOL FROM symbolsmapped WHERE recoName = '"+sym+"'"
    df3 = df3.append(pd.read_sql(sqlrun, con=database_connection),ignore_index=True)

In [12]:
len(df1)


3369

In [13]:
df4 =pd.concat([df1,df3],axis=1)
df4

,broker,company_name,current_price,recomended_buying,predict_date,target_price,SYMBOL
0,Choice Equity Broking,Karnataka Bank,75.80,110.55,2018-07-24,167.00,KTKBANK
1,IIFL,REC,143.00,126.26,2018-03-20,250.00,RECLTD
2,IIFL,NBCC (India),34.70,98.13,2018-03-20,200.00,NBCC
3,IIFL,SAIL,32.55,71.2,2018-03-20,150.00,SAIL
4,IIFL,ITC,243.70,258.68,2018-03-20,400.00,ITC
...,...,...,...,...,...,...,...
3365,Angel Broking,Capital First,587.50,695.05,2018-01-02,850.00,VIDEOIND
3366,Reliance Securities,Navkar Corporation,34.50,190.2,2018-01-02,229.00,CAPF
3367,Motilal Oswal,Titan Co,1040.90,848.19,2018-01-02,975.00,NAVKARCORP
3368,Axis Direct,IG Petrochem.,142.00,749.06,2018-01-02,832.00,TITAN


In [14]:
uniqueBrokers = df4.broker.unique()
uniqueBrokers

array(['Choice Equity Broking', 'IIFL', 'Rudra Shares & Stocks Brokers',
       'Indiabulls Ventures', 'Ventura', 'Motilal Oswal',
       'ICICIdirect.com', 'Centrum', 'JM Financial',
       'Geojit Financial Services', 'Sharekhan', 'HDFC Securities',
       'Arihant Capital Markets', 'Reliance Securities',
       'Kotak Securities', 'Axis Direct', 'Prabhudas Lilladher',
       'Dalmia Securities', 'AnandRathi', 'Dolat Capital', 'GEPL Capital',
       'KRChoksey', 'East India Securities Ltd', 'Edelweiss',
       'LKP Securities Limited', 'Cholamandalam Securities',
       'Elara Capital', 'Investec Securities', 'Nirmal Bang',
       'SPA Securities', 'CD Equisearch', 'AUM Capital', 'Angel Broking',
       'Ajcon Global', 'Joindre Capital Services', 'Religare',
       'Karvy Stock Broking', 'Bonanza', 'SMC Research',
       'Narnolia Financial Advisors', 'Phillip Capital', 'Dynamic Levels',
       'YES Securities', 'Hem Securities', 'Antique Stock Broking',
       'BP Wealth', 'Khambatt

In [15]:
#del brokerPerformace
brokerPerformace = pd.DataFrame(columns=['Name','Hit Ratio','Avg. Growth','Total Recos'])
brokerPerformace

,Name,Hit Ratio,Avg. Growth,Total Recos


In [17]:
for brokerName in uniqueBrokers :
    dfbrokerRecos = df4.loc[df4['broker'] == brokerName]
    print('brokerName : ',brokerName)
    hits=0
    growthSum=0
    total=0
    for index, row in dfbrokerRecos.iterrows():
        #print(row)
        sym = row['SYMBOL']
        print(type(sym))

        avgGrowth=0
        targetp=0
        recbuying=0
        maxval=0
        growthperc=0
        
        if type(sym) == str :
            mydate1 = row['predict_date']#prediction date
            mydate2 = mydate1 + relativedelta(years=1)#date 1 year after prediction
            dbyear1 = "nse"+str(mydate1.year)
            dbyear2 = "nse"+str(mydate2.year)
        
            sqlrun1 = "SELECT * FROM "+dbyear1+" WHERE Date >= '"+str(mydate1)+"' AND Symbol = '"+sym+"'"
            dfprices1 = pd.read_sql(sqlrun1, con=database_connection)
            del dfprices1['index']
        
            sqlrun2 = "SELECT * FROM "+dbyear2+" WHERE Date <= '"+str(mydate2)+"' AND Symbol = '"+sym+"'"
            dfprices2 = pd.read_sql(sqlrun2, con=database_connection)
            del dfprices2['index']
            dfprices = pd.concat([dfprices1,dfprices2],ignore_index=True)

            targetp = float(row['target_price'])
        
            print("SYMBOL : ",sym)
            dfhits = dfprices.loc[dfprices['Closeval'] >= targetp]
            
            total = total + 1
            
            if not dfhits.empty :
                print("HIT!")
                hits = hits + 1
                recbuying = float(row['recomended_buying'])
                maxval = dfprices['Closeval'].max()
                growthperc = (maxval - recbuying) *100 / recbuying
                print('Growth : ',growthperc)
                if np.isnan(growthperc):
                    total = total-1
                else:    
                    if(growthperc < 200):
                        growthSum = growthSum + growthperc
                    else:
                        print("----------------ALERT--------------")
                        high = recbuying * 1.2
                        low = recbuying * 0.8
                        recDate = row['predict_date']
                        print("recDate : ",recDate)
                        dferr = dfprices.loc[dfprices['Date'] == recDate]
                        if not dferr.empty :
                            if dferr['Closeval'].iloc[0] >= low and dferr['Closeval'].iloc[0] <= high :
                                growthSum = growthSum + growthperc
                            else:
                                total = total - 1
                                hits = hits - 1
                        else:
                            total = total - 1
                            hits = hits - 1
            else:
                print("DIDN'T HIT!")
            del dfprices
            del dfhits
    if total != 0:
        hitRatio = int(hits*100 / total)
        avgGrowth = int(growthSum / total)
        print("Broker : ", brokerName, "HitRatio : ", hitRatio, "Avg. Growth : ", avgGrowth, "Total Recos. : ", total)
        brokerPerformace = brokerPerformace.append({'Name':brokerName,'Hit Ratio':hitRatio,'Avg. Growth':avgGrowth,'Total Recos':total}, ignore_index=True)

brokerName :  Choice Equity Broking
<class 'str'>
SYMBOL :  KTKBANK
DIDN'T HIT!
<class 'str'>
SYMBOL :  AXISBANK
HIT!
Growth :  38.621200889547815
<class 'str'>
SYMBOL :  HDFCBANK
HIT!
Growth :  14.762769945493424
<class 'str'>
SYMBOL :  INDUSINDBK
DIDN'T HIT!
<class 'str'>
SYMBOL :  TEJASNET
DIDN'T HIT!
<class 'str'>
SYMBOL :  WIPRO
DIDN'T HIT!
<class 'str'>
SYMBOL :  ISFT
HIT!
Growth :  274.6935795128566
----------------ALERT--------------
recDate :  2018-03-15
<class 'str'>


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:58: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and 'the values will not compare equal to the
'datetime.date'. To retain the current behavior, convert the
'datetime.date' to a datetime with 'pd.Timestamp'.


SYMBOL :  CUMMINSIND
HIT!
Growth :  53.62400014002415
<class 'str'>
SYMBOL :  MANAPPURAM
HIT!
Growth :  1110.1201823456279
----------------ALERT--------------
recDate :  2018-03-07
<class 'str'>
SYMBOL :  VIVIMEDLAB
DIDN'T HIT!
<class 'str'>
SYMBOL :  JKLAKSHMI
HIT!
Growth :  33.366920973081896
<class 'str'>
SYMBOL :  KOTAKBANK
HIT!
Growth :  102.01230619182168
<class 'str'>
SYMBOL :  GODREJCP
HIT!
Growth :  813.8191429110817
----------------ALERT--------------
recDate :  2018-01-20
<class 'str'>
SYMBOL :  DEN
HIT!
Growth :  302.3328149300155
----------------ALERT--------------
recDate :  2018-01-18
Broker :  Choice Equity Broking HitRatio :  54 Avg. Growth :  122 Total Recos. :  11
brokerName :  IIFL
<class 'str'>
SYMBOL :  RECLTD
HIT!
Growth :  1275.7326152383969
----------------ALERT--------------
recDate :  2018-03-20
<class 'str'>
SYMBOL :  NBCC
HIT!
Growth :  118.23091816977481
<class 'str'>
SYMBOL :  SAIL
DIDN'T HIT!
<class 'str'>
SYMBOL :  ITC
DIDN'T HIT!
<class 'str'>
SYMBOL :

SYMBOL :  BRITANNIA
DIDN'T HIT!
<class 'str'>
SYMBOL :  HINDALCO
DIDN'T HIT!
<class 'str'>
SYMBOL :  BIRLACORPN
DIDN'T HIT!
<class 'str'>
SYMBOL :  PIIND
HIT!
Growth :  56.845854922279784
<class 'str'>
SYMBOL :  NILKAMAL
DIDN'T HIT!
<class 'str'>
SYMBOL :  ENIL
DIDN'T HIT!
<class 'str'>
SYMBOL :  ICIL
DIDN'T HIT!
<class 'str'>
SYMBOL :  ARVIND
DIDN'T HIT!
<class 'str'>
SYMBOL :  MARUTI
DIDN'T HIT!
<class 'str'>
SYMBOL :  LAURUSLABS
DIDN'T HIT!
<class 'str'>
SYMBOL :  BSE
DIDN'T HIT!
<class 'str'>
SYMBOL :  DIVISLAB
HIT!
Growth :  45.92148760330579
<class 'str'>
SYMBOL :  TATAPOWER
HIT!
Growth :  13.98784866059099
<class 'str'>
SYMBOL :  JKLAKSHMI
HIT!
Growth :  20.794026365116927
<class 'str'>
SYMBOL :  MARICO
HIT!
Growth :  11.77271180673698
<class 'str'>
SYMBOL :  MRPL
DIDN'T HIT!
<class 'str'>
SYMBOL :  HEROMOTOCO
DIDN'T HIT!
<class 'str'>
SYMBOL :  RAMCOCEM
HIT!
Growth :  22.68433770227722
<class 'str'>
SYMBOL :  TITAN
HIT!
Growth :  44.47619666168734
<class 'str'>
SYMBOL :  KEC
DI

SYMBOL :  SUNPHARMA
HIT!
Growth :  29.966808004451167
<class 'str'>
SYMBOL :  HINDUNILVR
HIT!
Growth :  18.280356007628733
<class 'str'>
SYMBOL :  PRSMJOHNSN
DIDN'T HIT!
<class 'str'>
SYMBOL :  NHPC
HIT!
Growth :  6496.5200764818355
----------------ALERT--------------
recDate :  2018-06-05
<class 'str'>
SYMBOL :  TATAMOTORS
DIDN'T HIT!
<class 'str'>
SYMBOL :  KNRCON
DIDN'T HIT!
<class 'str'>
SYMBOL :  BLUESTARCO
HIT!
Growth :  14.737890319368363
<class 'str'>
SYMBOL :  SAIL
HIT!
Growth :  17.34559789750329
<class 'str'>
SYMBOL :  ALKEM
DIDN'T HIT!
<class 'str'>
SYMBOL :  UBL
HIT!
Growth :  25.437966988897795
<class 'str'>
SYMBOL :  ONGC
DIDN'T HIT!
<class 'str'>
SYMBOL :  POWERGRID
DIDN'T HIT!
<class 'str'>
SYMBOL :  NAUKRI
HIT!
Growth :  68.48859315589351
<class 'str'>
SYMBOL :  AEGISCHEM
DIDN'T HIT!
<class 'str'>
SYMBOL :  ASHOKA
HIT!
Growth :  46.76810639190079
<class 'str'>
SYMBOL :  SADBHAV
DIDN'T HIT!
<class 'str'>
SYMBOL :  IPCALAB
HIT!
Growth :  46.03257232096378
<class 'str'>


...

SYMBOL :  CEATLTD
DIDN'T HIT!
<class 'str'>
SYMBOL :  SIEMENS
HIT!
Growth :  5.268959435626102
<class 'str'>
SYMBOL :  ESSELPACK
HIT!
Growth :  101.13895216400911
<class 'str'>
SYMBOL :  INFRATEL
DIDN'T HIT!
<class 'str'>
SYMBOL :  HEROMOTOCO
DIDN'T HIT!
<class 'str'>
SYMBOL :  SRTRANSFIN
DIDN'T HIT!
<class 'str'>
SYMBOL :  UPL
HIT!
Growth :  98.93597392213545
<class 'str'>
SYMBOL :  MARUTI
DIDN'T HIT!
<class 'str'>
SYMBOL :  SHOPERSTOP
DIDN'T HIT!
<class 'str'>
SYMBOL :  EQUITAS
DIDN'T HIT!
<class 'str'>
SYMBOL :  HINDZINC
DIDN'T HIT!
<class 'str'>
SYMBOL :  IDEA
DIDN'T HIT!
<class 'str'>
SYMBOL :  HDFC
DIDN'T HIT!
<class 'str'>
SYMBOL :  RALLIS
DIDN'T HIT!
<class 'str'>
SYMBOL :  MCX
DIDN'T HIT!
<class 'str'>
SYMBOL :  KOTAKBANK
HIT!
Growth :  16.367166522080403
<class 'str'>
SYMBOL :  SHREECEM
DIDN'T HIT!
<class 'str'>
SYMBOL :  STRTECH
DIDN'T HIT!
<class 'str'>
SYMBOL :  YESBANK
DIDN'T HIT!
<class 'str'>
SYMBOL :  TEJASNET
DIDN'T HIT!
<class 'str'>
SYMBOL :  SHRIRAMCIT
DIDN'T HIT!


SYMBOL :  MANAPPURAM
HIT!
Growth :  1429.607124148769
----------------ALERT--------------
recDate :  2018-02-09
<class 'str'>
SYMBOL :  HEIDELBERG
DIDN'T HIT!
<class 'str'>
SYMBOL :  GSPL
DIDN'T HIT!
<class 'str'>
SYMBOL :  DISHTV
DIDN'T HIT!
<class 'NoneType'>
<class 'str'>
SYMBOL :  THERMAX
HIT!
Growth :  1518.2059800664451
----------------ALERT--------------
recDate :  2018-02-08
<class 'str'>
SYMBOL :  DISHTV
DIDN'T HIT!
<class 'str'>
SYMBOL :  IGL
DIDN'T HIT!
<class 'str'>
SYMBOL :  COCHINSHIP
HIT!
Growth :  80.56856187290968
<class 'str'>
SYMBOL :  CENTURYPLY
DIDN'T HIT!
<class 'str'>
SYMBOL :  RAMCOCEM
DIDN'T HIT!
<class 'str'>
SYMBOL :  SIEMENS
DIDN'T HIT!
<class 'str'>
SYMBOL :  JKCEMENT
HIT!
Growth :  107.39043824701194
<class 'str'>
SYMBOL :  GRINDWELL
DIDN'T HIT!
<class 'str'>
SYMBOL :  AIAENG
HIT!
Growth :  1167.2569444444443
----------------ALERT--------------
recDate :  2018-02-07
<class 'str'>
SYMBOL :  RECLTD
HIT!
Growth :  982.5802430663759
----------------ALERT------

SYMBOL :  IDFCFIRSTB
DIDN'T HIT!
<class 'str'>
SYMBOL :  CAPF
HIT!
Growth :  56.82492581602373
<class 'str'>
SYMBOL :  SHREECEM
HIT!
Growth :  3652.1850142394183
----------------ALERT--------------
recDate :  2018-01-12
<class 'str'>
SYMBOL :  HMVL
DIDN'T HIT!
<class 'str'>
SYMBOL :  SHREECEM
HIT!
Growth :  1270.2661024243293
----------------ALERT--------------
recDate :  2018-01-12
<class 'str'>
SYMBOL :  TCS
HIT!
Growth :  112.15707052867067
<class 'str'>
SYMBOL :  NMDC
DIDN'T HIT!
<class 'str'>
SYMBOL :  HSCL
HIT!
Growth :  496.9040247678019
----------------ALERT--------------
recDate :  2018-01-11
<class 'str'>
SYMBOL :  SRTRANSFIN
HIT!
Growth :  575.7003542861316
----------------ALERT--------------
recDate :  2018-01-08
<class 'str'>
SYMBOL :  COROMANDEL
HIT!
Growth :  264.6056782334385
----------------ALERT--------------
recDate :  2018-01-08
<class 'str'>
SYMBOL :  NMDC
DIDN'T HIT!
<class 'str'>
SYMBOL :  AXISBANK
DIDN'T HIT!
Broker :  Edelweiss HitRatio :  33 Avg. Growth :  55 

SYMBOL :  YESBANK
DIDN'T HIT!
<class 'str'>
SYMBOL :  CANBK
DIDN'T HIT!
<class 'str'>
SYMBOL :  ASHOKLEY
DIDN'T HIT!
<class 'str'>
SYMBOL :  ADLABS
DIDN'T HIT!
<class 'str'>
SYMBOL :  AJANTPHARM
DIDN'T HIT!
<class 'str'>
SYMBOL :  YESBANK
DIDN'T HIT!
<class 'str'>
SYMBOL :  SUNPHARMA
DIDN'T HIT!
<class 'str'>
SYMBOL :  BHARATFIN
HIT!
Growth :  754.3304347826086
----------------ALERT--------------
recDate :  2018-02-27
<class 'str'>
SYMBOL :  KNRCON
HIT!
Growth :  155.80218068535825
<class 'str'>
SYMBOL :  PNCINFRA
DIDN'T HIT!
<class 'str'>
SYMBOL :  PAGEIND
HIT!
Growth :  63967.52669039145
----------------ALERT--------------
recDate :  2018-02-23
<class 'str'>
SYMBOL :  ICIL
DIDN'T HIT!
<class 'str'>
SYMBOL :  ENIL
HIT!
Growth :  492.7355187778485
----------------ALERT--------------
recDate :  2018-02-06
<class 'str'>
SYMBOL :  J&KBANK
DIDN'T HIT!
<class 'str'>
SYMBOL :  VEDL
DIDN'T HIT!
Broker :  SPA Securities HitRatio :  7 Avg. Growth :  11 Total Recos. :  13
brokerName :  CD Equise

SYMBOL :  GREENPLY
DIDN'T HIT!
<class 'str'>
SYMBOL :  GAIL
HIT!
Growth :  80.64220183486239
<class 'str'>
SYMBOL :  TEAMLEASE
HIT!
Growth :  544.2952354504702
----------------ALERT--------------
recDate :  2018-03-20
<class 'str'>
SYMBOL :  LAURUSLABS
DIDN'T HIT!
Broker :  YES Securities HitRatio :  33 Avg. Growth :  26 Total Recos. :  3
brokerName :  Hem Securities
<class 'str'>
SYMBOL :  HDFCBANK
DIDN'T HIT!
<class 'str'>
SYMBOL :  BEPL
DIDN'T HIT!
Broker :  Hem Securities HitRatio :  0 Avg. Growth :  0 Total Recos. :  2
brokerName :  Antique Stock Broking
<class 'str'>
SYMBOL :  NIITTECH
HIT!
Growth :  36.85183383597625
<class 'str'>
SYMBOL :  HCLTECH
HIT!
Growth :  18.210878577493627
Broker :  Antique Stock Broking HitRatio :  100 Avg. Growth :  27 Total Recos. :  2
brokerName :  BP Wealth
<class 'str'>
SYMBOL :  MOIL
DIDN'T HIT!
Broker :  BP Wealth HitRatio :  0 Avg. Growth :  0 Total Recos. :  1
brokerName :  Khambatta Securities
<class 'str'>
SYMBOL :  KARDA
DIDN'T HIT!
<class 

In [18]:
brokerPerformace

,Name,Hit Ratio,Avg. Growth,Total Recos
0,Choice Equity Broking,54,122,11
1,IIFL,50,70,28
2,Rudra Shares & Stocks Brokers,0,0,2
3,Indiabulls Ventures,42,213,49
4,Ventura,11,9,9
5,Motilal Oswal,36,63,526
6,ICICIdirect.com,46,31,309
7,Centrum,39,20,51
8,JM Financial,36,30,95
9,Geojit Financial Services,50,29,64


In [19]:
brokerPerformace.to_sql(con=database_connection, name='brokerperformance2018', if_exists='replace')